In [1]:
import requests
import re
from bs4 import BeautifulSoup

url = 'https://pagina12.com.ar'
p12 = requests.get(url)
p12.status_code

200

In [2]:
print(p12.text)

a></span></div></div></div></article></div></div></div></div></div></div></div></div></div><div class="wrapper"><div class="tema-1"><div class="layout-region tema producto tradicional"><div class="row headlines-block headline-block-bg"><div class="headline-block-title"><div class="title-box"><div class="title-box_title"><a href="https://octubretv.com/" target="_blank"><img src="/assets/media/logos/logo_octubre_tv_n.svg" /></a></div><div class="title-box_btn"><a href="https://octubretv.com/" target="_blank"><div class="show-for-medium">Ver más</div><div class="hide-for-medium">Ver más</div></a></div></div></div><div class="headline-block-inner"><div class="columns small-12 medium-4 index-0"> <div class="headline-card headline-1 "><article class="headline-card-inner"><div class="block-multimedia"><div class="multimedia-container"><a href="https://octubretv.com/videos/ciclo-especia-documentales-mexicanos/" target="_blank"><div class="intrinsic-container"><img class="show-for-small-only  l

In [3]:
print(f'Encabezados de la pagina:\n{p12.headers}\n\n')
print(f'Encabezados de requests:\n{p12.request.headers}\n\n')
print(f'Metodo usado:\n{p12.request.method}\n\n')
print(f'url usada (Util cuando la pagina nos redirecciona):\n{p12.request.url}')


Encabezados de la pagina:
{'Date': 'Tue, 14 Apr 2020 18:58:13 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d6de1889e0c9083e3817aba6456363e541586890693; expires=Thu, 14-May-20 18:58:13 GMT; path=/; domain=.pagina12.com.ar; HttpOnly; SameSite=Lax', 'Vary': 'Accept-Encoding', 'X-DNS-Prefetch-Control': 'off', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'X-Backend': 'prod_frontend_1', 'X-Backend-TTL': '180.000', 'X-Type': 'Dynamic URI', 'Age': '0', 'grace': '86400.000 none', 'ttl': '119.841', 'x-debug': '', 'X-Instance': 'cache-front-prod-varnish-76c5f88cf6-xmgzh', 'x-restarts': '0', 'X-Cache': 'HIT (3)', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '583f

In [4]:
soup_p12 = BeautifulSoup(p12.text, 'lxml')
secciones = soup_p12.find(class_='hot-sections')('a')
secciones_links = {link.get_text() : link['href']  for link in secciones}

secciones_links

{'El país': 'https://www.pagina12.com.ar/secciones/el-pais',
 'Economía': 'https://www.pagina12.com.ar/secciones/economia',
 'Sociedad': 'https://www.pagina12.com.ar/secciones/sociedad',
 'Cultura y Espectáculos': 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'El mundo': 'https://www.pagina12.com.ar/secciones/el-mundo',
 'Deportes': 'https://www.pagina12.com.ar/secciones/deportes',
 'Plástica': 'https://www.pagina12.com.ar/secciones/plastica',
 'Contratapa': 'https://www.pagina12.com.ar/secciones/contratapa'}

In [5]:
def conectar_pagina(url):
    try:
        respons = requests.get(url)
    except Exception as e:
        print(f'Error:\n  {e}')
        return None
    if respons.status_code != 200:
        return None

    return respons

In [6]:
def obtener_links_articulos(seccion):

    sec = conectar_pagina(secciones_links[seccion])
    if sec:
        soup_seccion = BeautifulSoup(sec.text, 'lxml')

        article_list = soup_seccion.select('.featured-article__container h2 a, .article-box__container h2 a')
        articles = {article.get_text() : article.get('href') for article in article_list}

        print(f'Se han agregado {len(articles)} links de la seccion:\t{seccion}')

        return articles
    else: 
        return None

In [7]:
articulos = {}
for seccion in secciones_links:
    articulos.update(obtener_links_articulos(seccion))

for titulo, url in articulos.items():
    print(f'{titulo}\n{url}\n\n')

-en-la-provincia-de-buen


Coronavirus: primera reunión del equipo que administrará la cuarentena|Recibirán propuestas de los gobernadores
https://www.pagina12.com.ar/259592-coronavirus-primera-reunion-del-equipo-que-administrara-la-c


La Cámara de Diputados en tiempos de coronavirus: trabajo a distancia y reuniones virtuales|En 15 días 8 ministros expusieron ante los legisladores
https://www.pagina12.com.ar/259591-la-camara-de-diputados-en-tiempos-de-coronavirus-trabajo-a-d


Capitalismo e interrogantes
https://www.pagina12.com.ar/259583-capitalismo-e-interrogantes


Duro cruce entre Iglesias y Lipovetzky por el rol de la oposición durante la pandemia|“Goriglesias” y “Liporocotó”, segundo round en Twitter
https://www.pagina12.com.ar/259582-duro-cruce-entre-iglesias-y-lipovetzky-por-el-rol-de-la-opos


Coronavirus: La Cámara Federal de Casación recomendó prisión domiciliaria a presos no violentos  |Para prevenir la propagación del virus
https://www.pagina12.com.ar/259579-coronavirus-l

In [8]:
def datos_articulo(url):
    req_articulo = conectar_pagina(url)
    if not req_articulo:
        return None

    soup_articulo = BeautifulSoup(req_articulo.text, 'lxml')

    articulo = {}

    titulo =    soup_articulo.select_one('.article-title')
    fecha =     soup_articulo.select_one('.time span')
    autor =     soup_articulo.select_one('.article-author a')
    resumen =   soup_articulo.select_one('.article-summary')
    epigrafe =  soup_articulo.select_one('.article-main-media-text-image')
    imagen =    soup_articulo.select_one('.article-main-media-image__container .show-for-large-only')
    contenido = soup_articulo.select    ('.article-text p')

    articulo['titulo'] =    titulo.text             if titulo       else None
    articulo['fecha'] =     fecha.get('datetime')   if fecha        else None
    articulo['autor'] =     autor.get_text()        if autor        else None
    articulo['resumen'] =   resumen.get_text()      if resumen      else None
    articulo['epigrafe'] =  epigrafe.get_text()     if epigrafe     else None
    articulo['imagen'] =    imagen.get('data-src')  if imagen       else None
    articulo['contenido'] = contenido               if contenido    else None

    # for key, value in articulo.items():
    #     print(f'{key}\n{value}\n\n')

    return articulo


In [9]:
articulos_datos = []
urls = list(articulos.values())

In [10]:
for i in range(len(urls) - 1):
    url = urls[i]
    print(f"Extrayendo Articulo { i + 1 } de { len(articulos) - 1 - 1 }\n\t{url}")
    
    articulos_datos.append(datos_articulo(url))

Extrayendo Articulo 1 de 109
	https://www.pagina12.com.ar/259606-permisos-para-circulacion-la-nueva-lista-completa-y-como-con
Extrayendo Articulo 2 de 109
	https://www.pagina12.com.ar/259595-sera-obligatorio-el-uso-de-tapabocas-en-la-provincia-de-buen
Extrayendo Articulo 3 de 109
	https://www.pagina12.com.ar/259592-coronavirus-primera-reunion-del-equipo-que-administrara-la-c
Extrayendo Articulo 4 de 109
	https://www.pagina12.com.ar/259591-la-camara-de-diputados-en-tiempos-de-coronavirus-trabajo-a-d
Extrayendo Articulo 5 de 109
	https://www.pagina12.com.ar/259583-capitalismo-e-interrogantes
Extrayendo Articulo 6 de 109
	https://www.pagina12.com.ar/259582-duro-cruce-entre-iglesias-y-lipovetzky-por-el-rol-de-la-opos
Extrayendo Articulo 7 de 109
	https://www.pagina12.com.ar/259579-coronavirus-la-camara-federal-de-casacion-recomendo-prision-
Extrayendo Articulo 8 de 109
	https://www.pagina12.com.ar/259564-cuando-empiezan-las-clases-el-retorno-a-las-aulas-quedaria-p
Extrayendo Articulo 9 de 

In [11]:
import pandas as pd
df = pd.DataFrame(articulos_datos)
df.head()

,titulo,fecha,autor,resumen,epigrafe,imagen,contenido
0,Permisos para circulación: La nueva lista comp...,2020-04-14,Yumber Vera Rojas,None,"Permisos de circulación, lista completa",https://images.pagina12.com.ar/styles/focal_3_...,"[[Como establece , [Decreto 297/2020], \n, las..."
1,Será obligatorio el uso de tapabocas en la pro...,2020-04-14,Yumber Vera Rojas,Habrá multas para quienes incumplan la medida ...,None,https://images.pagina12.com.ar/styles/focal_3_...,"[[Desde el próximo lunes 20 de abril , [será o..."
2,Coronavirus: primera reunión del equipo que ad...,2020-04-14,Yumber Vera Rojas,El gobierno nacional decidió cómo estará integ...,El gobierno nacional publicó una resolución a...,https://images.pagina12.com.ar/styles/focal_3_...,[[La cuarentena administrada comienza a formal...
3,La Cámara de Diputados en tiempos de coronavir...,2020-04-14,Yumber Vera Rojas,None,None,https://images.pagina12.com.ar/styles/focal_3_...,"[[El , [presidente de la Cámara de Diputados, ..."
4,Capitalismo e interrogantes,2020-04-14,Jorge Alemán,None,None,https://images.pagina12.com.ar/styles/focal_3_...,"[[[Hasta ahora, el capitalismo mundial sólo se..."


In [12]:
df.to_csv('articulos pagina12.csv')